This Jupter notebook was used to process robot LRF data collected in training environment, in order to train and test neural network used in glass classification.

When rerunning it, just pay attention to the places marked with "<---- change the dirctory"

In [1]:
%matplotlib qt5 

from __future__ import print_function
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numpy import linspace, meshgrid
from matplotlib.mlab import griddata
from matplotlib import cm
from tqdm import tqdm
import math
import time

from sklearn.neural_network import MLPClassifier
# from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.externals import joblib

In [29]:
# Read in info from old npy files

"""
There are two sets of npy files, old and new, of same structure
old npy:
- x, y coordinates are based on the map build on sight using gmapping while doing experiment
- also, distance and theta were calculated based on the robot pose from same gmapping
- the last colum, glass prob, was produced by the SII version NN trained using data from B14F10
- modified gmapping (with occupancy from 0 to 100) produces map using different coordinate system
- so the old data cannot be checked using map built by new gmapping
- but the old npy has to be kept, because new NN's training data is from it
- the old maps can be retrieved by running original rosbags with unmodified gmapping

new npy:
- x, y are based on maps built by modified gmapping with changed parameters
- maps can be generated by running "gmapping_rosbag.launch" + rosbag starting with "modi_"
- glass prob is produced by new NN trained with interpolated glass data
"""

def loadInfo(path):
    mainPath = "/home/jiang/Desktop/Codes and Data/tests/npys/" # <---- change the dirctory
    info = np.load(mainPath+path)  # shape (<1081*n, 6)
    info[:,3]=np.rad2deg(info[:,3])
    print(path, "shape", info.shape)
    print("Max incident angle = %.2f deg" %np.amax(info[:,3]))
    print("%.2f %% of incident is within 90 deg\n"%(len(info[info[:,3]<90])/float(len(info))*100.0))
    return info

# data from robot gmapping environments 
# [0.sensorCoord_X 1.sensorCoord-Y 2.senRange 3.incidAngle 4.senIntensity 5.glassProb]

# old npy
B3F2Info = loadInfo("/npy/allGridInfoB3F2.npy")  # shape (<1081*n, 6)
B11F2Info = loadInfo("npy/allGridInfoB11F2.npy")  # shape (<1081*n, 6)
B14F10Info = loadInfo("npy/allGridInfoB14F10.npy")  # shape (<1081*n, 6)

# new npy
newB14F10Info = loadInfo("newNpy/newAllGridInfoB14F10.npy")
newB3F1Info = loadInfo("newNpy/newAllGridInfoB3F1(NN2).npy")
newB3F2Info = loadInfo("newNpy/newAllGridInfoB3F2(NN2).npy")
newB2F1GlassInfo = loadInfo("newNpy/newAllGridInfoB2F1GlassPart(NN2).npy")
newB11F3Info = loadInfo("newNpy/newAllGridInfoB11F3.npy")
newB11F2Info = loadInfo("newNpy/newAllGridInfoB11F2.npy")

# samples from the well controlled environment [dis, ang, intensity]
glassPlaneInfo = np.load("/home/jiang/Desktop/Codes and Data/tests/npys/npy/glassPlaneInfo.npy")  # shape (n, 3)   # <---- change the dirctory
glassPlaneInfo = abs(glassPlaneInfo)
glassPlaneInfo = glassPlaneInfo[glassPlaneInfo[:,0]>=1,:]
print("Original glassPlaneInfo.shape", glassPlaneInfo.shape)
print("The max incident angle in deg = ", np.amax(glassPlaneInfo[:,1]))

cardboardInfo = np.load("/home/jiang/Desktop/Codes and Data/tests/npys/npy/cardboardInfo.npy")  # shape (n, 3)     # <---- change the dirctory
cardboardInfo = abs(cardboardInfo)
print("\nOriginal cardboardInfo.shape", cardboardInfo.shape)
print("The max incident angle in deg = ", np.amax(cardboardInfo[:,1]))

silverInfo = np.load("/home/jiang/Desktop/Codes and Data/tests/npys/npy/silverInfo.npy")  # shape (n,3)           # <---- change the dirctory
silverInfo = abs(silverInfo)
print("\nOriginal silverInfo.shape", silverInfo.shape)
print("The max incident angle in deg = ", np.amax(silverInfo[:,1]))

/npy/allGridInfoB3F2.npy shape (464873, 6)
Max incident angle = 93.75 deg
99.98 % of incident is within 90 deg

npy/allGridInfoB11F2.npy shape (403682, 6)
Max incident angle = 96.50 deg
99.79 % of incident is within 90 deg

npy/allGridInfoB14F10.npy shape (877616, 6)
Max incident angle = 270.00 deg
99.89 % of incident is within 90 deg

newNpy/newAllGridInfoB14F10.npy shape (890613, 6)
Max incident angle = 266.50 deg
99.88 % of incident is within 90 deg

newNpy/newAllGridInfoB3F1(NN2).npy shape (445352, 6)
Max incident angle = 128.00 deg
99.86 % of incident is within 90 deg

newNpy/newAllGridInfoB3F2(NN2).npy shape (426750, 6)
Max incident angle = 270.00 deg
99.96 % of incident is within 90 deg

newNpy/newAllGridInfoB2F1GlassPart(NN2).npy shape (238730, 6)
Max incident angle = 266.00 deg
99.90 % of incident is within 90 deg

newNpy/newAllGridInfoB11F3.npy shape (161645, 6)
Max incident angle = 270.00 deg
99.83 % of incident is within 90 deg

newNpy/newAllGridInfoB11F2.npy shape (451846,

In [3]:
# Define functions related to taking data from experiment and building training dataset

def FindInfo(info, p1, p2, dw=0.1):
    """
    Input
    -----
    info:   array.shape = (pNum, 6)
    x1, y1: P1, starting point
    x2, y2: P2, ending point
    dw: tolerance distance to the line formed by P1 and P2
    
    Return
    ------
    Info of points whose distance to the line formed by P2 and P2 is less than dw
    """
    x1, y1 = p1[0], p1[1]
    x2, y2 = p2[0], p2[1]
    x0= info[:,0]
    y0= info[:,1]
    minY, maxY = min(y1, y2)-dw, max(y1, y2)+dw
    dist = abs((y2-y1)*x0-(x2-x1)*y0+x2*y1-y2*x1)/np.sqrt((y2-y1)*(y2-y1)+(x2-x1)*(x2-x1)) # dist.shape (pNum,)
    return info[(dist<dw)&(y0>minY)& (y0<maxY)]

def RemovePoint(info, p0, dp=0.15):
    x0, y0 = p0[0], p0[1]
    x, y = info[:,0], info[:,1]
    return info[~((x>x0-dp)&(x<x0+dp)&(y>y0-dp)&(y<y0+dp))] # to be improved: use circle instead of square

def RvMultiPoints(info, p1, p2, Num, dp=0.15):
    x1, y1 = p1[0], p1[1]
    x2, y2 = p2[0], p2[1]
    x0= info[:,0]
    y0= info[:,1]
    xs = np.linspace(x1, x2, num=Num)
    ys = np.linspace(y1, y2, num=Num)
    for x, y in zip(xs, ys):
        info = RemovePoint(info, (x, y), dp)
    return info

def FindInfoRvPoints(allInfo, p1, p2, Num, dp=0.15, dw=0.1):
    """
    Aim: to extract glass info and remove evenly distributed frames' info
    Input: 
    -----
    info: all info of the environment
    p1: start point of the line
    p2: end point of the line
    Num: number of frames on this glass line
    dp: how large the frame's area is (half of the square or circle)
    dw: tolerance distance to the glass line
    """
    info = FindInfo(allInfo, p1, p2, dw)
    info = RvMultiPoints(info, p1, p2, Num, dp)
    return info


def TakeEvenInfoSpace(inInfo, maxDis=20, maxAng=90, maxInte=20000, dGap=0.1, aGap=1, iGap=100, num=1):
    """
    Aim
    ---
    1. take the points evenly for training dataset
    2. dividing the distance-angle-intensity space evenly into many small grids
    3. then take "num" points in each grid
    4. if there is no enough points in current grid, take all existing points 
  
    Input
    -----
    inInfo: info set need to be filtered
    maxDis, maxAng, maxInte: max distance, max angle and max intensity, values larger than which will be ignored
    dGap, aGap, iGap: grid's length in distance, angle direction and intensity
    num: number of points to take in each grid
    
    Output
    ------
    Array.shape = (n, 6), all points taken in above way
    
    """
    outInfo = np.array([]).reshape(0,6)
    disEdge = np.arange(1, maxDis, dGap)
    angEdge = np.arange(0, maxAng, aGap)
    inteEdge = np.arange(0, maxInte, iGap)
    dis = inInfo[:,2]
    
    for de in disEdge:
        subInfo1 = inInfo[(dis>=de)&(dis<de+dGap),:]
        if len(subInfo1)>0:
            subAng = subInfo1[:,3]
            maxSubAng = np.amax(subAng)
            for ae in angEdge[angEdge<maxSubAng]:
                subInfo2 = subInfo1[(subAng>=ae)&(subAng<ae+aGap),:]
                if len(subInfo2)>0:
                    subInte = subInfo2[:,4]
                    maxSubInte = np.amax(subInte)
                    for ie in inteEdge[inteEdge<maxSubInte]:
                        allCandidate = subInfo2[(subInte>ie)&(subInte<ie+iGap),:]
                        candNum = allCandidate.shape[0]
                        if candNum >= num:
                            ind = np.random.randint(0,candNum,size=num)
                            choice = allCandidate[ind]
                            outInfo = np.concatenate((outInfo,choice),axis=0)
                        if 0<candNum<num:
                            outInfo = np.concatenate((outInfo,allCandidate),axis=0)
    print("Take out %d points evenly from original %d point dataset, rate = %.2f %%" 
          %(len(outInfo), len(inInfo), float(len(outInfo))/len(inInfo)) )
    return outInfo

def TakeEvenInfoSurface(inInfo, maxDis=20, maxAng=90, dGap=0.1, aGap=1, num=1):
    """    
    Aim
    ---
    Similar function with former one, 
    just only dividing the distance-angle space evenly into many small grids 
    """
    inteRange = 8000
    return TakeEvenInfoSpace(inInfo, maxDis=maxDis, maxAng=maxAng, maxInte=inteRange, 
                             dGap=0.1, aGap=1, iGap=inteRange, num=1)

In [4]:
# Define function: plot info's 3d feature

def PlotFeature3D(inInfo, lim=(20, 100, 20000), showMap=False, mInfo=B3F2Info):
    """
    Aim: plot 3d distribution of [distance, angle, intensity] dataset
    
    Input
    -----
    inInfo: list of different info, info's shape can be (n, 6) or (n, 3)
            eg: [allGlassInfo, cEvenInfo, bEvenInfo]     allGlassInfo.shape=(n,6)
            each info list will be plotted in different colors, max list number is 4, but can be extended
    lim: x,y,z axis max length of the plot
    showMap: whether show location of input info on map 
    mInfo: array name of the map's scan data, by default B3F2 map
    
    """
    # check and plot glassInfo
    if len(inInfo)<=0: 
        print("Please input [distance, angle, intensity] info")
        return
    elif len(inInfo)>4:
        print("Please input less than 4 sets of info!")
        return
    
    # build the 3d plot object and assign some proprities
    fig = plt.figure("3D Features")
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('Distance')
    ax.set_ylabel('Incident angle')
    ax.set_zlabel('Intensity')
    ax.set_xlim(0,lim[0])
    ax.set_ylim(0,lim[1])
    ax.set_zlim(0,lim[2])
    ax.set_yticks(np.arange(0,lim[1],20))
    colors = ["r", "b", "y", "g"] # if want to extend input list number, add more color here
    n = 0
    
    if showMap:
        # Plot out the glass map and sample area
        fig = plt.figure("Samples in Map")
        ax1 = fig.add_subplot(111)
        ax1.scatter(mInfo[:,0], mInfo[:,1], marker=".", s=0.5, color = "gray")
        ax1.set_xlim(np.amin(mInfo[:,0])-5,np.amax(mInfo[:,0])+5)
        ax1.set_ylim(np.amin(mInfo[:,1])-5,np.amax(mInfo[:,1])+5)
        ax1.set_aspect("equal")

    
    for info, c in zip(inInfo, colors):
        n+=1
        print("Info %d shape ="%n, info.shape)
        
        disInd = 0
        angInd = 1
        inteInd = 2
        
        if info.shape[1] == 6:
            disInd = 2
            angInd = 3
            inteInd = 4
            
            if showMap: 
                ax1.scatter(info[:,0], info[:,1], marker=".",s=1, color = c)
                print("show info %d in the map" %n)

        ax.scatter(info[:,disInd], info[:,angInd], info[:,inteInd], marker='.', s=1.5, color=c)
        
    plt.show()
    return

In [5]:
# Define interpolate function

def InterpoByBoundry(inInfo, maxDis=20, maxAng=90, maxInte=20000, dGap=0.1, aGap=1.0, iGap=100.0, num=1, debug=False):
    """
    Aim: interpolate the data based on the upper bounder of the data
    Input:
    info: shape = (n, 3)
    maxDis=20, maxAng=90, maxInte=20000: the max range of each dimension
    dGap=0.1, aGap=1.0, iGap=100.0: the gap or stride in each dimension
    num: the aiming number of points in each divided block or sub-space
    """
    outInfo = np.copy(inInfo)
        
    # find the max dis, ang, inte in inInfo
    maxDisIn = np.amax(inInfo[:,0])
    maxAngIn = np.amax(inInfo[:,1])
    maxInteIn = np.amax(inInfo[:,2])
    
    # build the occupancy recording array
    dN = int(math.ceil((min(maxDis, maxDisIn)-1)/dGap))
    aN = int(math.ceil(min(maxAng, maxAngIn)/aGap))
    iN = int(math.ceil(min(maxInte, maxInteIn)/iGap))
    
    occup = np.zeros((dN, aN, iN))
    occup2 = np.copy(occup)
    if debug == True: print("occup shape =", occup.shape)
    
    # register occup array according inInfo
    disEdge = np.arange(1, maxDisIn, dGap)
    angEdge = np.arange(0, maxAngIn, aGap)
    inteEdge = np.arange(0, maxInteIn, iGap)
    dis = inInfo[:,0]
    
    for de in disEdge:
        subInfo1 = inInfo[(dis>=de)&(dis<de+dGap),:]
        if len(subInfo1)>0:
            dn = int(math.floor((de-1)/dGap))
            subAng = subInfo1[:,1]
            maxSubAng = np.amax(subAng)
            
            for ae in angEdge[angEdge<maxSubAng]:
                subInfo2 = subInfo1[(subAng>=ae)&(subAng<ae+aGap),:]
                if len(subInfo2)>0:
                    an = int(math.floor(ae/aGap))
                    subInte = subInfo2[:,2]
                    maxSubInte = np.amax(subInte)
                    
                    for ie in inteEdge[inteEdge<maxSubInte]:
                        subInfo3 = subInfo2[(subInte>ie)&(subInte<ie+iGap),:]
                        candNum = len(subInfo3)
                        if candNum != 0: 
                            inn = int(math.floor(ie/iGap))
                            occup[dn,an,inn]=1

    # find the upper boundry surface 
    for i in np.arange(iN):
        for a in np.arange(aN):
            if len(np.nonzero(occup[:,a,i])[0]) != 0: 
                maxd = np.amax(np.nonzero(occup[:,a,i]))

                occup2[:maxd+1,:a+1,:i+1]=1
    
    # interpolate according to upper boundry
    occup3 = occup2-occup
    NumBeIntpo = np.unique(occup3, return_counts=True)[1][1]
    
    intpoIndD, intpoIndA, intpoIndI = np.nonzero(occup3)
    randD, randA, randI = np.random.rand(3, len(intpoIndD))
      
    intpoD = (intpoIndD+randD)*dGap+1.0
    intpoA = (intpoIndA+randA)*aGap
    intpoI = (intpoIndI+randI)*iGap
    
    nePoints = np.c_[intpoD, intpoA, intpoI]
    outInfo = np.concatenate((outInfo, nePoints), axis=0)
    print("\nResults of interpolation: \nReal points = %d, Interpolated points = %d, Total points = %d, Interpolation rate = %.2f %%"
          %(len(inInfo), NumBeIntpo, len(outInfo), 100.0*NumBeIntpo/len(outInfo)))
    
    return outInfo

In [6]:
""" 
Extract training data from each info array (old rosbag)

Training dataset from Bldg3Floor2 and Bldg11Floor2

[0.sensorCoord_X 1.sensorCoord-Y 2.senRange 3.incidAngle 4.senIntensity 5.glassProb]

point = (x, y); line = (p1, p2)

"""

### black metal from Bldg3Floor2
p1, p2 = (-14.82, 35.395), (-15.784, 31.246) # wall on left-top
p3, p4 = (-16.404, 7.8983), (-18.142, 10.676) # wall behind glass
p5, p6 = (-12.304, 30.086), (-12.012, 31.284) # pillar (left)
p7, p8 = (-10.878, 31.021), (-11.182, 29.839)
p9, p10 = (-7.0632, 28.849), (-6.7951, 30.041) # pillar (right)
p11, p12 = (-5.6404, 29.746), (-5.9217, 28.565)

blackMetal = [[p1, p2], # black metal wall on left-top
        [p3, p4], # black metal wall behind glass
        [p5, p6], [p6, p7], [p7, p8], [p8, p5], # black metal pillar (left)
        [p9, p10], [p10, p11], [p11, p12], [p12, p9]] # black metal pillar (right)

blackMetalInfo = np.array([]).reshape(0,6)
for l in blackMetal: 
    blackMetalInfo = np.concatenate((blackMetalInfo, FindInfo(B3F2Info, l[0], l[1])), axis = 0)

    
### concrete wall from Bldg3Floor2
p13, p14 = (-2.9765, -0.4782), (4.6488, -2.367)
p15, p16 = (-20.255, 25.643), (-20.488, 24.837)
p17, p18 = (-0.76371, 21.15), (-0.98061, 20.188)
p19, p20 = (-2.5009, 14.062), (-2.6744, 13.298)
p21, p22 = (-5.7403, 0.20702), (-4.9207, 0.00015843)
p23, p24 = (-11.897, 1.1485), (-10.997, 0.92948)
p25, p26 = (-13.817, 4.3382), (-13.34, 3.5531)
p27, p28 = (-15.818, 7.5822), (-15.349, 6.834)
p29, p30 = (-18.676, 12.17), (-18.155, 11.374)
p31, p32 = (-21.957, 18.722), (-22.174, 17.873)
p33, p34 = (-18.671, 32.422), (-18.805, 31.757)
p35, p36 = (-20.654, 15.399), (-20.194, 14.698)
p37, p38, p39 = (-4.1544, 7.1659),(-4.3563, 6.2665),(-3.38, 6.0182)
p40, p41 = (4.8057, 3.969),(6.1035, 3.6039)
p42, p43 = (8.5084, 3.0254),(10.057, 2.627)
p44, p45 = (8.7727, -3.3765),(7.0546, -2.9886)
p46, p47 = (-3.1876, 9.168), (5.7004, 6.9367) # to be imporved: remove the noisy peak near 0 deg

concreteWall = [[p13, p14], [p15, p16], [p17, p18], [p19, p20], [p21, p22], [p23, p24], 
          [p25, p26], [p27, p28], [p29, p30], [p31, p32], [p33, p34], [p35, p36],
          [p37, p38], [p38, p39], [p40, p41], [p42, p43],[p43, p44], [p44, p45], [p46, p47]]
              
concreteInfo = np.array([]).reshape(0,6)
for l in concreteWall: 
    concreteInfo = np.concatenate((concreteInfo, FindInfo(B3F2Info, l[0], l[1])), axis = 0)
    
### white wall from Bldg11Floor2 (only this is from new rosbags)
p1, p2 = (5.2203, -1.3304), (2.763, 2.672)
p3, p4 = (4.9253, -2.8348), (-1.5999, -6.882)

whiteWall = [[p1, p2], [p3, p4]]

whiteWallInfo = np.array([]).reshape(0,6)
for l in whiteWall:
    whiteWallInfo = np.concatenate((whiteWallInfo, FindInfo(newB11F2Info, l[0], l[1])), axis = 0)

    
### all non-glass information
allNonGlassInfo = np.concatenate((concreteInfo, blackMetalInfo), axis=0)

# take part of the data because too much
cInfo = concreteInfo[np.arange(0, len(concreteInfo), 10),:]
bInfo = blackMetalInfo[np.arange(0, len(blackMetalInfo), 10), :]
ngInfo = allNonGlassInfo[np.arange(0,len(allNonGlassInfo),10),:]
   
    
### clean glass info from Bldg3Floor2
p48, p49 = (-16.565, 15.803), (-11.145, 7.0148)
p50, p51 = (-14.264, 5.1113), (-19.689, 13.83)
p52, p53 = (-22.045, 18.732), (-20.588, 24.644)
p54, p55 = (-20.293, 25.75), (-18.92, 31.662)
p56, p57 = (-18.595, 32.451), (-16.188, 31.881)
p58, p59 =(-12.01, 34.734), (-4.1985, 32.915)
p60, p61 = (0.1446, 24.894), (-0.69784, 21.249)
p62, p63 = (-1.5078, 17.87), (-1.8168, 16.491)
p64, p65 = (-3.0254, 11.65), (-4.0946, 7.2395)
p66, p67 =  (4.7753, -2.448), (6.9033, -2.973)
p68, p69 = (-12.002, 1.0597), (-13.314, 3.4077)

# remove noise 
tempInfo = FindInfoRvPoints(B3F2Info,  p52, p53, 9, dw=0.1)
glass1 = tempInfo[(tempInfo[:,2]<5)&(tempInfo[:,3]<20)]
tempInfo = FindInfoRvPoints(B3F2Info,  p54, p55, 9, dp=0.2, dw=0.1)
glass2 = tempInfo[tempInfo[:,3]<3] # need to clean further
tempInfo = FindInfoRvPoints(B3F2Info,  p51, p48, 5, dp=0.1, dw=0.1) 
glass3 = tempInfo[tempInfo[:,3]<20]
tempInfo = FindInfoRvPoints(B3F2Info,  p49, p48, 13, dp=0.15, dw=0.1)
glass4 = tempInfo
tempInfo = FindInfoRvPoints(B3F2Info,  p49, p48, 13, dp=0.15, dw=0.1) 
glass5 = tempInfo
tempInfo = FindInfoRvPoints(B3F2Info,  p60, p61, 6, dp=0.2, dw=0.1) # some noise difficult to be cleaned
glass6 = tempInfo[tempInfo[:,3]<20]
tempInfo = FindInfoRvPoints(B3F2Info,  p62, p63, 3, dp=0.2, dw=0.1) 
glass7 = tempInfo
tempInfo = FindInfoRvPoints(B3F2Info,  p64, p65, 7, dp=0.2, dw=0.1)
glass8 = tempInfo[tempInfo[:,3]<20]
tempInfo = FindInfoRvPoints(B3F2Info, p66, p67, 2,  dp=0.2, dw=0.1) 
glass9 =  tempInfo[(tempInfo[:,3]<10)&(tempInfo[:,4]<2000)]
   
cleanGlassInfo = np.concatenate((glass1, glass2, glass3, glass4, glass5, glass6, 
                                 glass7, glass8, glass9), axis =0)


### dirty glass from Bldg11Floor3
p1, p2 = (-2.8951, -5.3054), (-2.2947, -6.2156)
p3, p4 = (-0.58939, 4.0354), (5.4244, 7.9562)
glass10 = FindInfo(B11F2Info, p1, p2)
glass11 = FindInfoRvPoints(B11F2Info, p3, p4, 9)
glass12 = FindInfoRvPoints(B11F2Info, (2.7476, 6.1751), (2.6573, 6.1272), 0)

dirtyGlass = np.concatenate((glass10, glass11, glass12), axis=0)
dirtyGlassInfo = dirtyGlass[~((dirtyGlass[:,3]>30)&(dirtyGlass[:,4]>500))]


### All glass info (clean and dirty)
allGlassInfo = np.concatenate((cleanGlassInfo, dirtyGlassInfo), axis=0)


### old training sample from Bldg14Floor10 (for comparision)
p1, p2 = (5.0175, 4.3146),(8.3023, 5.0184)
oldWB3F2Info = FindInfo(B14F10Info, p1,p2)
p3, p4 = (22.84, 8.6602), (27.429, 9.7045)
oldGlassInfo = FindInfoRvPoints(B14F10Info, p3, p4, 6, dp=0.3)
oldGlassInfo = oldGlassInfo[~((oldGlassInfo[:,3]>30)&(oldGlassInfo[:,4]>500))]

In [7]:
# Prepare dataset for training and testing NN

# evenly take data for training neural network
cEvenInfo = TakeEvenInfoSurface(concreteInfo)
bEvenInfo = TakeEvenInfoSurface(blackMetalInfo,maxDis=15)
wEvenInfo = TakeEvenInfoSurface(whiteWallInfo)
gEvenInfo = TakeEvenInfoSpace(allGlassInfo)

# only keep 3-d feature info
cEvenInfo3 = cEvenInfo[:,2:5]
bEvenInfo3 = bEvenInfo[:,2:5]
wEvenInfo3 = wEvenInfo[:,2:5]
gEvenInfo3 = gEvenInfo[:,2:5]
# np.save("/home/jiang/Desktop/evenGlass.npy", gEvenInfo3)

# interpolate glass data
interpoEvenGlass = InterpoByBoundry(gEvenInfo3, maxAng=90, dGap=1, aGap=1.0, iGap=150.0, num=1, debug=True)

# interpolate data for Neural network
glassForNN = interpoEvenGlass  # to be improved: add glassPlaneInfo? (finally didn't use in training)
nonGlassForNN = np.concatenate((cEvenInfo3, bEvenInfo3, wEvenInfo3), axis=0)

Take out 9465 points evenly from original 95379 point dataset, rate = 0.10 %
Take out 5642 points evenly from original 63893 point dataset, rate = 0.09 %
Take out 4983 points evenly from original 48848 point dataset, rate = 0.10 %
Take out 3847 points evenly from original 18063 point dataset, rate = 0.21 %
occup shape = (18, 89, 132)

Results of interpolation: 
Real points = 3847, Interpolated points = 9282, Total points = 13129, Interpolation rate = 70.70 %


In [8]:
# Function for training & testing and using neural network

def BuildClassifier(glassData, nonGlassData, method="NN", testRate=0.2, NNsize=(10, 10), plot=False):
    """
    Aim: train and test neural network using input data, show test result
    Input
    -----
    glassData, nonGlassData: array.shape=(n,3) [distance, angle, intensity]
    testRate: percentage of data used for testing
    NNsize: shape of the neural network
    
    Output
    -----
    the object of the trained neural network and scalar
    
    """
    
    # prepare training and testing data
    gLabel = np.full((glassData.shape[0],), 1.0)
    ngLabel = np.full((nonGlassData.shape[0],), 0.0)
    
    xData = np.concatenate((glassData, nonGlassData), axis=0)
    yData = np.concatenate((gLabel, ngLabel), axis=0)
    print("Glass = %d, nonGlass = %d, total = %d, rate = %.2f : 1" 
          %(len(glassData), len(nonGlassData), len(xData), float(len(glassData))/len(nonGlassData)))
    
    # split train and test data, and scale them
    trainx, testx, trainy, testy = train_test_split(xData, yData, test_size = testRate)
    scalar = preprocessing.MinMaxScaler()
    scalar.fit(trainx)
    trainx = scalar.transform(trainx)
    testx = scalar.transform(testx)
    
    # choose classifier type
    if method == "NN":
        # train and test NN
        clf = MLPClassifier(hidden_layer_sizes=NNsize)
    
    if method == "SVM":
        clf = svm.SVC(kernel="rbf")

    # train and evaluate classifier
    t0=time.time()
    clf.fit(trainx, trainy)
    t1 = time.time()-t0
    clf.predict(testx)
    t2 = time.time()-t1-t0
    print("%s score: %f, train time: %f s, predict time: %f s per 1000 samples" 
          %(method, clf.score(testx, testy), t1, 1000*t2/len(testx)))
    
    # classification result evaluation
    gPredIn = scalar.transform(glassData)
    ngPredIn = scalar.transform(nonGlassData)
    gPredOut = clf.predict(gPredIn)
    ngPredOut = clf.predict(ngPredIn)
    corGlass = glassData[gPredOut>=0.5]
    wrgGlass = glassData[gPredOut<0.5]
    corNonGlass = nonGlassData[ngPredOut<0.5]
    wrgNonGlass = nonGlassData[ngPredOut>=0.5]
    
    if plot == True:
        PlotFeature3D([corGlass, corNonGlass, wrgGlass, wrgNonGlass])
        
    return clf, scalar  


def classifierNN(GlassNN, Scaler, scanInfo):
    scanInfo = Scaler.transform(scanInfo)
    probs = GlassNN.predict_proba(scanInfo) # probs.shape (<1081,2) (nonGlassProb, glassProb)
    glassProb = probs[:,1] # glassProb.shape (<1081,)
    return glassProb

In [19]:
### train NN glass classifier and save results to files

"""
History records for training NN

NN1: gEvenInfo(only real data), nonGlassForNN (concrete+blackMetal), method="NN", testRate=0.2, NNsize=(10,10)
NN2: glassForNN (real + interpoByBoundry), nonGlassForNN (concrete+blackMetal), method="NN", testRate=0.2, NNsize=(10,10)
NN3: glassForNN (real + interpoByBoundry), nonGlassForNN (concrete+blackMetal+whiteWall), method="NN", testRate=0.2, NNsize=(10,10)
"""
NN, scalerNN = BuildClassifier(gEvenInfo[:,2:5], nonGlassForNN, method="NN", testRate=0.2, NNsize=(5,5), plot=True)
# SVM, scalerSVM = BuildClassifier(gEvenInfo[:,2:5], nonGlassForNN, method="SVM", testRate=0.2, NNsize=(10,10), plot=True)

### save trained NN and its scalar
# joblib.dump(clf3, "/home/jiang/catkin_ws/src/tests/pkls/NN3.pkl")
# joblib.dump(scaler3, "/home/jiang/catkin_ws/src/tests/pkls/Scaler3.pkl")

### load NN and its scaler
# GlassNN =joblib.load("/home/jiang/catkin_ws/src/tests/pkls/NN3.pkl")
# Scaler=joblib.load("/home/jiang/catkin_ws/src/tests/pkls/Scaler3.pkl")

Glass = 3847, nonGlass = 20090, total = 23937, rate = 0.19 : 1
NN score: 0.945280, train time: 2.440996 s, predict time: 0.000117 s per 1000 samples
Info 1 shape = (2842, 3)
Info 2 shape = (19701, 3)
Info 3 shape = (1005, 3)
Info 4 shape = (389, 3)


In [20]:
# Plot out certain points' 3D distribution

def updateProb( prior, evidence):
    """update the glassMap based on info of each scan based on Bayes rules (just get average before)"""
    newProb = prior * evidence / (prior*evidence + (1-prior)*(1-evidence))
    return newProb

def plotPrediInfo(targInfo, inputInfo, d=0.2, classifier=GlassNN, scaler=Scaler):
    
    print("targInfo.shape ", targInfo.shape, "; Incorrect angle: ", len(targInfo[targInfo[:,3]<-0.8]))
    if len(targInfo)==0:
        return
    
    # use NN to predict 
    prob = classifierNN(GlassNN, Scaler, targInfo[:,2:5])

    # take out the glass and noglass info and calculate final proba
    glass = targInfo[prob>=0.5,:]
    noglass = targInfo[prob<0.5,:]
    post =0.5
    for p in prob:
#         print("before update: \tpost %f, \tp %f" %(post, p))
        post = updateProb(post, p)
#         print("after update: \tpost %.6f, \tp %.6f" %(post, p))
    print("Prediction results: glass %d, non-glass %d, final proba %.6f" %(len(glass), len(noglass), post))
    
    # plot out glass and nonglass info
    fig = plt.figure("Prediction Results")
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(glass[:,2], glass[:,3], glass[:,4],marker='.', color='r')
    ax.scatter(noglass[:,2], noglass[:,3], noglass[:,4],marker='.', color='b')

    # setting related to plot
    ax.set_xlabel('Distance')
    ax.set_ylabel('Incident angle')
    ax.set_zlabel('Intensity')
    ax.set_xlim(0,np.amax(inputInfo[:,2])+1)
    ax.set_ylim(-5, 100)
    ax.set_zlim(0,np.amax(inputInfo[:,4])+1000)
    ax.set_yticks(np.arange(0,100,20))
    
def plotOnePoint(p1, inputInfo, d=0.2,):
    """
    Aim: plot the 3-d feature of a single point
    Input:
    -----
    targX, targY: x, y coordinate of target point in the map
    """
    targX=p1[0]
    targY=p1[1]
    x = inputInfo[:,0]
    y = inputInfo[:,1]
    targInfo = inputInfo[(targX-d<=x)&(x<=targX+d)&(targY-d<=y)&(y<=targY+d),:]
    plotPrediInfo(targInfo, inputInfo, d)
    
    return targInfo

def plotOneLine(p1, p2, inputInfo, dw=0.2):
    
    targInfo = FindInfo(inputInfo, p1, p2, dw=dw)
    plotPrediInfo(targInfo, inputInfo, dw)
    
    return targInfo

newB114F10Info = loadInfo("newNpy/newAllGridInfoB14F10.npy")
checkInfo = newB14F10Info
p1, p2 = (-27.572, -3.2611), (-25.173, -5.0457)
p3, p4 = (4.9253, -2.8348), (-1.5999, -6.882)
# targInfo1 = plotOnePoint(p1, checkInfo, d=0.1) 
targInfo2 = plotOnePoint(p2, checkInfo, d=0.1)
# targInfo3 = plotOneLine( p1, p2, checkInfo,dw=0.1) 
# targInfo4 = plotOneLine( p3, p4, checkInfo,dw=0.1) 

newNpy/newAllGridInfoB14F10.npy shape (890613, 6)
Max incident angle = 266.50 deg
99.88 % of incident is within 90 deg

targInfo.shape  (491, 6) ; Incorrect angle:  0
Prediction results: glass 14, non-glass 477, final proba 0.000000


In [22]:
# use NN to predict 
data = FindInfo(B3F2Info,p13, p14)
re = classifierNN(GlassNN, Scaler, data[:,2:5])
print("percent of data classified as glass = ", len(re[re>=0.5])*1.0/len(re))

percent of data classified as glass =  0.0588403313339


In [23]:
# plot histogram bars in 3d 

testInfo = cEvenInfo
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
disEdge = np.arange(0, 20, 0.5)
angEdge = np.arange(0, 90, 2)
hist, xedges, yedges = np.histogram2d(testInfo[:,2], testInfo[:,3], bins=[disEdge, angEdge])

# Construct arrays for the anchor positions of the all bars.
# Note: np.meshgrid gives arrays in (ny, nx) so we use 'F' to flatten xpos,
# ypos in column-major order.
xpos, ypos = np.meshgrid(xedges[:-1] + 0.25, yedges[:-1] + 0.25)
xpos = xpos.flatten('F')
ypos = ypos.flatten('F')
zpos = np.zeros_like(xpos)

# Construct arrays with the dimensions for the all bars.
dx = 0.5 * np.ones_like(zpos)
dy = dx.copy()
dz = hist.flatten()

ax.bar3d(xpos, ypos, zpos, dx, dy, dz, color='b', zsort='average')

plt.show()